# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 10 2023 11:52AM,254656,15,30013106,"Alliance Pharma, Inc.",Released
1,Jan 10 2023 11:52AM,254656,15,30013107,"Alliance Pharma, Inc.",Released
2,Jan 10 2023 11:52AM,254656,15,30013108,"Alliance Pharma, Inc.",Released
3,Jan 10 2023 11:52AM,254656,15,30013109,"Alliance Pharma, Inc.",Released
4,Jan 10 2023 11:52AM,254656,15,30013110,"Alliance Pharma, Inc.",Released
5,Jan 10 2023 11:52AM,254656,15,30013111,"Alliance Pharma, Inc.",Released
6,Jan 10 2023 11:52AM,254656,15,30013112,"Alliance Pharma, Inc.",Released
7,Jan 10 2023 11:52AM,254656,15,30013113,"Alliance Pharma, Inc.",Released
8,Jan 10 2023 11:52AM,254656,15,30013114,"Alliance Pharma, Inc.",Released
9,Jan 10 2023 11:52AM,254656,15,30013115,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,254653,Released,1
26,254654,Released,1
27,254655,Released,1
28,254656,Released,11
29,254657,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254653,NaN,NaN,1.0
254654,NaN,NaN,1.0
254655,NaN,NaN,1.0
254656,NaN,NaN,11.0
254657,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254566,0.0,16.0,20.0
254567,0.0,0.0,10.0
254598,0.0,0.0,1.0
254599,0.0,0.0,1.0
254607,0.0,6.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254566,0,16,20
254567,0,0,10
254598,0,0,1
254599,0,0,1
254607,0,6,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254566,0,16,20
1,254567,0,0,10
2,254598,0,0,1
3,254599,0,0,1
4,254607,0,6,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254566,,16,20
1,254567,,,10
2,254598,,,1
3,254599,,,1
4,254607,,6,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc."
11,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc."
12,Jan 10 2023 11:40AM,254655,21,"NBTY Global, Inc."
13,Jan 10 2023 11:36AM,254654,10,ISDIN Corporation
14,Jan 10 2023 11:27AM,254653,20,Else Nutrition
15,Jan 10 2023 11:16AM,254652,10,MedStone Pharma LLC
22,Jan 10 2023 11:14AM,254651,10,"Senseonics, Incorporated"
23,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC"
41,Jan 10 2023 10:50AM,254649,15,"Alliance Pharma, Inc."
57,Jan 10 2023 10:48AM,254648,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",,,11
1,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",,,1
2,Jan 10 2023 11:40AM,254655,21,"NBTY Global, Inc.",,,1
3,Jan 10 2023 11:36AM,254654,10,ISDIN Corporation,,,1
4,Jan 10 2023 11:27AM,254653,20,Else Nutrition,,,1
5,Jan 10 2023 11:16AM,254652,10,MedStone Pharma LLC,,,7
6,Jan 10 2023 11:14AM,254651,10,"Senseonics, Incorporated",,,1
7,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC",,,18
8,Jan 10 2023 10:50AM,254649,15,"Alliance Pharma, Inc.",,,16
9,Jan 10 2023 10:48AM,254648,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",11,,
1,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",1,,
2,Jan 10 2023 11:40AM,254655,21,"NBTY Global, Inc.",1,,
3,Jan 10 2023 11:36AM,254654,10,ISDIN Corporation,1,,
4,Jan 10 2023 11:27AM,254653,20,Else Nutrition,1,,
5,Jan 10 2023 11:16AM,254652,10,MedStone Pharma LLC,7,,
6,Jan 10 2023 11:14AM,254651,10,"Senseonics, Incorporated",1,,
7,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC",18,,
8,Jan 10 2023 10:50AM,254649,15,"Alliance Pharma, Inc.",16,,
9,Jan 10 2023 10:48AM,254648,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",11,,
1,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",1,,
2,Jan 10 2023 11:40AM,254655,21,"NBTY Global, Inc.",1,,
3,Jan 10 2023 11:36AM,254654,10,ISDIN Corporation,1,,
4,Jan 10 2023 11:27AM,254653,20,Else Nutrition,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",11,NaN,NaN
1,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",1,NaN,NaN
2,Jan 10 2023 11:40AM,254655,21,"NBTY Global, Inc.",1,NaN,NaN
3,Jan 10 2023 11:36AM,254654,10,ISDIN Corporation,1,NaN,NaN
4,Jan 10 2023 11:27AM,254653,20,Else Nutrition,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",11,0.0,0.0
1,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",1,0.0,0.0
2,Jan 10 2023 11:40AM,254655,21,"NBTY Global, Inc.",1,0.0,0.0
3,Jan 10 2023 11:36AM,254654,10,ISDIN Corporation,1,0.0,0.0
4,Jan 10 2023 11:27AM,254653,20,Else Nutrition,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1782509,20,0.0,7.0
15,1782352,81,26.0,0.0
19,1273059,51,2.0,2.0
20,254653,1,0.0,0.0
21,1018509,4,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1782509,20,0.0,7.0
1,15,1782352,81,26.0,0.0
2,19,1273059,51,2.0,2.0
3,20,254653,1,0.0,0.0
4,21,1018509,4,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,20,0.0,7.0
1,15,81,26.0,0.0
2,19,51,2.0,2.0
3,20,1,0.0,0.0
4,21,4,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,20.0
1,15,Released,81.0
2,19,Released,51.0
3,20,Released,1.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,7.0,0.0,2.0,0.0,0.0
Executing,0.0,26.0,2.0,0.0,0.0
Released,20.0,81.0,51.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,7.0,0.0,2.0,0.0,0.0
1,Executing,0.0,26.0,2.0,0.0,0.0
2,Released,20.0,81.0,51.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,7.0,0.0,2.0,0.0,0.0
1,Executing,0.0,26.0,2.0,0.0,0.0
2,Released,20.0,81.0,51.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()